In [1]:
import platform
import os

system = platform.system().lower().strip()
if system in ['darwin', 'windows']:
    path_root = '.'
elif system == 'linux':
    from google.colab import drive
    drive.mount('/content/drive')

    path_root = './drive/MyDrive/code/03.crawler/39.OEMPartsOnline'
else:
    print('fuck')

path_root

Mounted at /content/drive


'./drive/MyDrive/code/03.crawler/39.OEMPartsOnline'

In [2]:
from gevent import monkey
monkey.patch_all(thread=False)

from gevent.pool import Pool
pool = Pool(300)

from gevent.lock import BoundedSemaphore
semaphore_correct, semaphore_error, semaphore_remain = BoundedSemaphore(1), BoundedSemaphore(1), BoundedSemaphore(1)

from IPython.display import clear_output
from datetime import datetime

# = = =  = = =  = = =

import pandas as pd

# = = =  = = =  = = =

from urllib.parse import urlparse, parse_qs, urlencode

import requests

import sys
sys.path.append(os.path.join(path_root, '../00.Tools'))
from crawler_configuration_oversea import get_header, get_proxy

from bs4 import BeautifulSoup
from lxml import etree

import re
import html as html_lib

# = = =  = = =  = = =

import math

# = = =  = = =  = = =

print('Data Loading...')
list_dict_correct, list_series_error, df_input = [], [], pd.read_excel(os.path.join(path_root, './file/input.xlsx'),
                                                                       header=0,
                                                                       dtype=str).fillna('')
print('Loading Done !')
print()

df_input['No'] = df_input['No'].astype(int)

crawler_remain = len(df_input)

print(f'总数量：{crawler_remain}')
print()

# = = =  = = =  = = =

def crawler(crawler_tuple):
    global list_dict_correct, list_series_error, crawler_remain

    crawler_status, (crawler_index, crawler_series) = False, crawler_tuple

    # = = =  = = =  = = =

    try:
        url_parse = urlparse(crawler_series['Url'])

        dict_param = parse_qs(url_parse.query)
        dict_param.pop('page', None)

        url_request = f'{url_parse.scheme}://{url_parse.netloc}{url_parse.path}?{urlencode(dict_param, doseq=True)}'

        crawler_retry = 0
        while True:
            crawler_retry += 1

            try:
                resp = requests.get(url_request,
                                    headers=get_header(),
                                    proxies=get_proxy(),
                                    timeout=(5, 15))

                if resp.status_code == 200:
                    soup = BeautifulSoup(resp.text, 'lxml')
                    html = etree.HTML(str(soup))

                    result = html.xpath('//h1[@data-test-id="page-subheader__page_heading"]')[0]
                    result = etree.tostring(result).decode('utf-8')
                    result = re.sub(r'<.*?>', '', result)
                    result = re.sub(r' +', ' ', result.strip())
                    result = html_lib.unescape(result)

                    count = int(result.split('Results')[0].replace(',', '').replace(' ', ''))
                    if 'results for' in result.lower() and dict_param['search_str'][0].lower().strip() in result.lower() and count > 0:
                        break
            except requests.exceptions.ProxyError as e:
                if '456 CN Client Forbidden' in str(e):
                    print('Long live the People\'s Republic of China!')
                    break
            except KeyboardInterrupt:
                break
            except:
                continue

        # = = =  = = =  = = =

        with semaphore_correct:
            list_dict_correct.append({'No': crawler_series['No'],
                                      'Url': url_request,
                                      'SKU Count': count,
                                      'Page': math.ceil(count / 18)})

        # = = =  = = =  = = =

        crawler_status = True
    except:
        crawler_series['Request_Url'] = url_request

        with semaphore_error:
            list_series_error.append(crawler_series)

    # = = =  = = =  = = =

    if crawler_remain % 30 == 0:
        clear_output()
    print(f'''[{crawler_status}] - [请求{crawler_retry}次] - [剩余{crawler_remain - 1}条] - [{datetime.now().strftime('%Y/%m/%d %H:%M:%S')}] - {crawler_series['No']}. {url_request}\n''')

    with semaphore_remain:
        crawler_remain -= 1

# = = =  = = =  = = =

pool.map(crawler, df_input.iterrows())

print('Data Outputting...')
if list_dict_correct:
    df_correct = pd.DataFrame(list_dict_correct).sort_values(by=['No'],
                                                             ascending=[True],
                                                             ignore_index=True)
    df_correct.to_excel(os.path.join(path_root, f'''./file/1.page-{datetime.now().strftime('%Y%m%d_%H%M%S')}.xlsx'''), index=False)
print('Output Finished !')
print()
if list_series_error:
    df_error = pd.DataFrame(list_series_error).sort_values(by=['No'],
                                                           ascending=[True],
                                                           ignore_index=True)
    df_error.to_excel(os.path.join(path_root, './file/page_error.xlsx'), index=False)
    print('- Error Occurred -')
    print()

# = = =  = = =  = = =

print('Done ~')

It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is be

Data Loading...


It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is be

Loading Done !

总数量：2



It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is be

[True] - [请求1次] - [剩余1条] - [2024/11/29 06:53:38] - 1. https://toyota.oempartsonline.com/search?search_str=Resonator



It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is being used.
Please set an environment variable with:
GEVENT_SUPPORT=True
to enable gevent support in the debugger.
It seems that the gevent monkey-patching is be

[True] - [请求1次] - [剩余0条] - [2024/11/29 06:53:40] - 2. https://nissan.oempartsonline.com/search?search_str=Resonator

Data Outputting...
Output Finished !

Done ~
